# Setup Minedojo Environment

In [1]:
from Environments import SkyRunner, MultithreadGym
frame_stack = False
frames = 4
use_grayscale=False

# Multithreaded environment wrapper
env = MultithreadGym.MultithreadGym(thread_int=1, env_int=1, frame_stack=frame_stack, frames_int=frames, use_grayscale=use_grayscale)

# Evaluation environment
eval_env = SkyRunner.CustomEnv(frame_stack=frame_stack, frames_int=frames, use_grayscale=use_grayscale)
eval_env.reset()

[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


starting Reloader 0
ThreadID: 0 has received an enviornment from queue. Reset of environement is being prepeared


[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.
[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


ThreadID 0 put complete environment into ready-queue.


array([[[ 34,  38,  41, ...,  45,  41,  38],
        [ 37,  40,  44, ...,  48,  45,  41],
        [ 40,  44,  48, ...,  51,  48,  44],
        ...,
        [ 61,  64,  68, ...,  67,  70,  66],
        [ 58,  61,  65, ...,  64,  61,  64],
        [ 56,  59,  62, ...,  62,  58,  61]],

       [[ 40,  45,  48, ...,  53,  49,  45],
        [ 44,  47,  52, ...,  56,  53,  48],
        [ 47,  51,  56, ...,  60,  56,  52],
        ...,
        [ 71,  76,  80, ...,  42,  44,  42],
        [ 68,  72,  77, ...,  40,  38,  40],
        [ 65,  69,  74, ...,  39,  37,  39]],

       [[ 51,  56,  60, ...,  67,  61,  56],
        [ 55,  60,  66, ...,  71,  66,  60],
        [ 60,  65,  71, ...,  76,  71,  66],
        ...,
        [ 90,  95, 100, ...,  29,  30,  29],
        [ 86,  91,  97, ...,  28,  27,  28],
        [ 83,  88,  93, ...,  27,  25,  27]]], dtype=uint8)

# Begin training

In [3]:
import train_openAI
import importlib

importlib.reload(train_openAI)

train_openAI.train(env, eval_env=eval_env, eval_freq=5000, use_prioritized_replay=False, name="Olav-init-leaves-2")

Waiting for ready enviornment
Ready enviornment received
Inventory and weather cleared!
Inventory and weather cleared!
Broken block detected. Moving to location 1
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Broken block detected. Moving to location 2
Inventory and weather cleared!
Broken block detected. Moving to location 3
Inventory and weather cleared!
Inventory and weather cleared!
Broken block detected. Moving to location 4
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Broken block detected. Moving to location 5
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Broken block detected. Moving to location 6
Inventory and weather cleared!
Broken block detected. Moving to location 7
Inventory and w

KeyboardInterrupt: 

# Shutdown Environments

In [4]:
env.close()
eval_env.close()

stopping Reloader 0


# Load existing model

In [ ]:
from stable_baselines3 import DQN
from CustomBaselines3.DoubleDQN import DoubleDQN

_env = env
model = DoubleDQN.load("/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/dDQN-checkpoints/checkpoint_30000_steps.zip")

obs = env.reset()
acc_r = 0
while True:
    act, st = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(act)

    acc_r += reward

    env.render()

    if done:
        obs = env.reset()
        print("Finished with reward %d" % acc_r)
        acc_r = 0